## Who is your GOLDEN GOOSE?

In [5]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [6]:
def load_csv(filename):
    orders = pd.read_csv(filename,sep=',')
    return orders
orders = load_csv("order.csv")

In [7]:
orders.head()

,order_date,order_id,customer,grand_total
0,9/7/11,CA-2011-100006,Dennis Kane,378
1,7/8/11,CA-2011-100090,Ed Braxton,699
2,3/14/11,CA-2011-100293,Neil Franzšsisch,91
3,1/29/11,CA-2011-100328,Jasper Cacioppo,4
4,4/8/11,CA-2011-100363,Jim Mitchum,21


In [8]:
import datetime as dt
NOW = dt.datetime(2014,12,31)
orders['order_date'] = pd.to_datetime(orders['order_date'])


In [9]:
rfmTable = orders.groupby('customer').agg({'order_date': lambda x: (NOW - x.max()).days, # Recency
                                        'order_id': lambda x:len(x),      # Frequency
                                        'grand_total': 'sum'}) # Monetary Value

rfmTable['order_date'] = rfmTable['order_date'].astype(int)
rfmTable.rename(columns={'order_date': 'recency', 
                         'order_id': 'frequency', 
                         'grand_total': 'monetary_value'}, inplace=True)
rfmTable = rfmTable.reset_index()

In [10]:
df = rfmTable.iloc[:,1:4]
df.head()

,recency,frequency,monetary_value
0,415,3,887
1,12,7,1744
2,88,7,3050
3,54,8,7756
4,34,10,3249


In [11]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
kmeans = KMeans(n_clusters=3)
kmeans.fit(df)
labels = kmeans.predict(df)
centroids = kmeans.cluster_centers_


In [12]:
centroids

array([[  1.00449612e+02,   7.59302326e+00,   4.20797674e+03],
       [  1.35833333e+02,   8.29629630e+00,   1.00412222e+04],
       [  1.72902287e+02,   5.40956341e+00,   1.39163617e+03]])

In [13]:
df['cluster_group'] = labels

In [10]:
df.head()

,recency,frequency,monetary_value,cluster_group
0,415,3,887,2
1,12,7,1744,2
2,88,7,3050,0
3,54,8,7756,1
4,34,10,3249,0


In [2]:
#!pip install git+https://github.com/jamalsenouci/causalimpact.git


  Cloning https://github.com/jamalsenouci/causalimpact.git to /private/var/folders/42/_1xq97bx1xbfgzvn58m_7cr4h987px/T/pip-req-build-b1p346bo
  Running setup.py bdist_wheel for causalimpact ... done
  Stored in directory: /private/var/folders/42/_1xq97bx1xbfgzvn58m_7cr4h987px/T/pip-ephem-wheel-cache-si45vzm4/wheels/30/c8/87/c89e20ccb85e26afb8e2f52f12560123610b44df483925083f
Successfully built causalimpact
You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [33]:
from causalimpact import CausalImpact
from statsmodels.tsa.arima_process import arma_generate_sample
data = df.loc[lambda df: df['cluster_group'] == 1]
data.head()

,recency,frequency,monetary_value,cluster_group
3,54,8,7756,1
6,41,10,14474,1
45,410,12,7888,1
75,307,4,11790,1
90,558,5,10502,1


In [34]:
import numpy as np
data["monetary_value_control"] = data["monetary_value"] - np.random.choice([1, 9, 20], data.shape[0])
data.head()

,recency,frequency,monetary_value,cluster_group,monetary_value_control
3,54,8,7756,1,7755
6,41,10,14474,1,14465
45,410,12,7888,1,7879
75,307,4,11790,1,11789
90,558,5,10502,1,10482


In [35]:
data = data[['recency','monetary_value','monetary_value_control']]
data.sort_values('recency',inplace=True)

In [36]:
data.set_index('recency')

,monetary_value,monetary_value_control
recency,,
2,8237,8228
4,7474,7465
6,8241,8232
9,12210,12209
9,8647,8627
16,7661,7652
21,9801,9781
29,7955,7946
33,7295,7286


In [37]:

pre_period = [0, 74]
post_period = [74, 835]

ci = CausalImpact(data, pre_period, post_period)


In [38]:
ci.plot()

AttributeError: 'CausalImpact' object has no attribute 'inferences'

<Figure size 1080x864 with 0 Axes>

In [41]:
ci.run()

KeyError: 0